In [27]:
# Install and update spaCy 
!pip install -U spacy

# Download the english language model
!python -m spacy download en

!pip install -U openpyxl

You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.
2023-06-05 12:23:52.041332: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-05 12:23:52.189503: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-05 12:23:52.194115: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-06-05 12:

In [5]:
import os
import csv
import pandas as pd
import numpy as np
import math

from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

In [6]:
version = 15

#Settings: remove # to select, add # to unselect

## Select classifier

###Logistic Regression
#classifier=LogisticRegression()
###Random Forest Classifier
#classifier = RandomForestClassifier()
###Linear Support Vector Classification
classifier = LinearSVC()
#

## Ignore tokens? 

###No
IgnoreLimit = 0
###Yes, ignore token with coef under 0.05
#IgnoreLimit = 0.005
#

## Ignore most frequent tokens?

###No
#max_df=1
###Ignore 0.5% most frequent tokens
#max_df=0.995
#



In [9]:
#Load the df generated in PrepareData
df = pd.read_csv('NewDf.csv')
del df[df.columns[0]]

df

,Filename,CGMech,content
0,ABB Group Annual Report 2015_English-2.txt,8,\n \n The ABB Group Annual Report 2015 \n \n \...
1,ABB_02.txt,8,ABB Group Annual Report 2002\n \n Financial re...
2,ABB_03.txt,8,Important information regarding the ABB 2003 A...
3,ABB_04.txt,8,ABB Annual Report 2004 \n \n Financial review ...
4,ABB_05.pdf.txt,9,i\n \n D\n e\n s\n g\n n\n e\n d\n \n \n \n b...
...,...,...,...
1569,WISeKey_2017.pdf.txt,9,ANNUAL REPORT 2017\n \n O I N \n \n - DIGI...
1570,WiSeKey-Annual-Report-2018.pdf.txt,9,Annual Report 2018\n \n ARCHITECTING A WISER W...
1571,ZurRoseGroup-2017-EN-Vollbericht.pdf.txt,9,Annual Report 2017\n \n Zur Rose Group \n \n ...
1572,ZurRoseGroup-2018-EN-Annual-Report.pdf.txt,9,Annual Report 2018\n \n Zur Rose Group \n \n ...


In [11]:
#Group CGMech by categories: 'low', 'middle', 'high'
def categorize(value):
    value=int(value)
    if value >= 1 and value <= 5:
        return 'low'
    elif value >= 6 and value <= 8:
        return 'middle'
    elif value >= 9 and value <= 10:
        return 'high'
    else:
        return 'unknown'

# apply function to create new "cat" column
df['cat'] = df['CGMech'].apply(categorize)

#split dataset: 80%train, 20% test
X = df.content
ylabels=df.CGMech
X_train, X_test, y_train, y_test = train_test_split(X, ylabels, test_size=0.2, random_state=1232)


In [12]:
#visualize categories
cat_counts = df['cat'].value_counts()

print(cat_counts)
cat_percentages = cat_counts / len(df) * 100
print('Total: ', len(df))
print()
cat_percentages=cat_percentages.round(0)

print("Percentage:")
print(cat_percentages)

#Baserate
dummy = DummyClassifier(strategy='most_frequent')
dummy.fit(None, y_train)
baserate = dummy.score(None, y_test)
print("Baserate: ")
print(baserate)

high      817
middle    619
low       138
Name: cat, dtype: int64
Total:  1574

Percentage:
high      52.0
middle    39.0
low        9.0
Name: cat, dtype: float64
Baserate: 
0.23809523809523808


In [13]:
X

0       \n \n The ABB Group Annual Report 2015 \n \n \...
1       ABB Group Annual Report 2002\n \n Financial re...
2       Important information regarding the ABB 2003 A...
3       ABB Annual Report 2004 \n \n Financial review ...
4       i\n \n D\n e\n s\n g\n n\n e\n d\n \n  \n \n b...
                              ...                        
1569    ANNUAL REPORT 2017\n \n O I N  \n \n   -  DIGI...
1570    Annual Report 2018\n \n ARCHITECTING A WISER W...
1571    Annual Report 2017\n \n Zur Rose Group \n \n 
...
1572    Annual Report 2018\n \n Zur Rose Group \n \n 
...
1573    Annual Report 2019\n \n Zur Rose Group \n \n 
...
Name: content, Length: 1574, dtype: object

In [14]:
import spacy

#tokenize string (content of .txt doc) to word tokens
def spacyTokens(SpacyString):
    sp = spacy.load('en_core_web_sm')  #load English language model
    sp.max_length=6000000 #increase max length of document (len(logitech09.pdf.txt) is over 5M)
    doc = sp(SpacyString)
    cleaned_tokens = []
    for token in doc:
        if not token.is_stop and token.is_alpha and not token.is_digit and not token.is_punct:
            cleaned_token = token.lemma_.lower().strip()
            if cleaned_token:
                cleaned_tokens.append(cleaned_token)
    return cleaned_tokens



/shared-libs/python3.9/py/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-06-05 08:34:50.255543: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-05 08:34:50.372238: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-05 08:34:50.377570: W tensorflow/stream_executor/platform/default/dso_loader.

In [16]:
pipe = Pipeline([
    ('vectorizer', TfidfVectorizer(tokenizer=spacyTokens)),
    ('classifier', LinearSVC())
])


In [17]:
print("fitting...")
pipe.fit(X_train, y_train)

fitting...


Pipeline(steps=[('vectorizer',
                 TfidfVectorizer(tokenizer=<function spacyTokens at 0x7f15fcc45160>)),
                ('classifier', LinearSVC())])

In [18]:
vectorizer = pipe.named_steps['vectorizer']
classifier = pipe.named_steps['classifier']
feature_names = vectorizer.get_feature_names()
coefs = classifier.coef_[0].tolist()



/shared-libs/python3.9/py/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [19]:
len(feature_names)

135858

In [20]:
len(coefs)

135858

In [21]:
# Extract tokens
all_tokens = [(feature_names[i], round(coefs[i], 2)) for i in range(len(feature_names))]
#all_tokens = [(feature_names[i], round(coefs[i], 2)) for i in range(len(feature_names)) if str(round(coefs[i],2)) != '0.0' and str(round(coefs[i],2)) != '-0.0']
print(all_tokens)

# Write all tokens to CSV
with open('all_tokens_'+str(version)+'.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Token', 'Coefficient'])
    for token, coef in all_tokens:
        writer.writerow([token, coef])

), ('schwierige', 0.0), ('schwierigem', 0.0), ('schwierigen', 0.0), ('schwieriger', -0.0), ('schwierigkeiten', 0.0), ('schwiertz', 0.0), ('schwieters', 0.0), ('schwiezer', -0.0), ('schwiiz', 0.0), ('schwiizerischer', 0.0), ('schwil', 0.0), ('schwindle', 0.0), ('schwindt', 0.0), ('schwingen', 0.0), ('schwirzer', 0.0), ('schwitzgebel', 0.0), ('schwiz', 0.0), ('schwizer', 0.0), ('schwizgebel', 0.0), ('schwob', 0.0), ('schwung', -0.0), ('schwyn', 0.0), ('schwynig', 0.0), ('schwyz', -0.0), ('schwyzer', 0.0), ('schwäbische', 0.0), ('schwäbischen', -0.0), ('schwäche', 0.0), ('schwächen', 0.0), ('schwächer', 0.0), ('schwächerer', 0.0), ('schwächte', 0.0), ('schwächung', 0.0), ('schwärzler', 0.0), ('schwörstadt', 0.0), ('schybig', 0.0), ('schymanietz', 0.0), ('schäble', 0.0), ('schädeli', 0.0), ('schädigung', -0.0), ('schädler', 0.0), ('schäfer', 0.0), ('schäft', 0.0), ('schäfts', -0.0), ('schäftsbericht', 0.0), ('schäftsjahr', 0.0), ('schäftsjahre', -0.0), ('schäftsleitungsteam', 0.0), ('schäf

In [22]:
#Positive
input_file = 'all_tokens_'+str(version)+'.csv'
output_file = 'top_positive_tokens_'+str(version)+'.csv'

# Read input CSV file and filter rows
filtered_rows = []
with open(input_file, 'r') as csvfile:
    reader = csv.reader(csvfile)
    next(reader)  # skip header row
    for row in reader:
        if float(row[1]) >= IgnoreLimit:
            filtered_rows.append(row)
print(filtered_rows)

# Write filtered rows to output CSV file
with open(output_file, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Token', 'Coefficient'])
    for row in filtered_rows:
        writer.writerow(row)


['sicence', '0.0'], ['sich', '-0.0'], ['siche', '0.0'], ['sichelschmidt', '0.0'], ['sicher', '0.0'], ['sichere', '0.0'], ['sicheren', '0.0'], ['sicherer', '0.0'], ['sichergestellt', '-0.0'], ['sichergestellten', '-0.0'], ['sicherheit', '-0.0'], ['sicherheits', '0.0'], ['sicherheitsansprüche', '0.0'], ['sicherheitsaspekte', '0.0'], ['sicherheitsbereich', '0.0'], ['sicherheitsdokumente', '0.0'], ['sicherheitsdruck', '0.0'], ['sicherheitsdrucke', '-0.0'], ['sicherheitsdruckerei', '0.0'], ['sicherheitsdruckprodukten', '-0.0'], ['sicherheitsdrucks', '-0.0'], ['sicherheitselektronik', '0.0'], ['sicherheitselement', '0.0'], ['sicherheitselemente', '0.0'], ['sicherheitsfond', '0.0'], ['sicherheitslösung', '0.0'], ['sicherheitsmerkmal', '0.0'], ['sicherheitsmerkmale', '0.0'], ['sicherheitsrelevanter', '0.0'], ['sicherheitstechnik', '0.0'], ['sicherheitstechnologie', '0.0'], ['sicherheitsüberwachung', '0.0'], ['sicherlich', '0.0'], ['sichern', '0.0'], ['sicherstellen', '0.0'], ['sicherstellt', '

In [23]:
#Negative
input_file = 'all_tokens_'+str(version)+'.csv'
output_file = 'top_negative_tokens_'+str(version)+'.csv'

# Read input CSV file and filter rows
filtered_rows = []
with open(input_file, 'r') as csvfile:
    reader = csv.reader(csvfile)
    next(reader)  # skip header row
    for row in reader:
        if float(row[1]) <= -IgnoreLimit:
            filtered_rows.append(row)
print(filtered_rows)


# Write filtered rows to output CSV file
with open(output_file, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Token', 'Coefficient'])
    for row in filtered_rows:
        writer.writerow(row)


], ['sichtigt', '-0.0'], ['sichtlich', '0.0'], ['sichtprüfung', '-0.0'], ['sichtweise', '0.0'], ['sichtweite', '0.0'], ['sichuan', '-0.0'], ['sician', '0.0'], ['sicilia', '0.0'], ['sicilian', '0.0'], ['siciliana', '0.0'], ['sicily', '-0.0'], ['sick', '-0.0'], ['sicker', '0.0'], ['sickkid', '0.0'], ['sickkids', '0.0'], ['sickle', '0.0'], ['sickness', '-0.0'], ['siclari', '0.0'], ['sicon', '0.0'], ['sicovam', '0.0'], ['sicpa', '-0.0'], ['sicta', '0.0'], ['sid', '0.0'], ['sidad', '0.0'], ['siddharth', '0.0'], ['siddle', '0.0'], ['side', '-0.0'], ['sideboard', '0.0'], ['sided', '-0.0'], ['sidekick', '0.0'], ['sidel', '0.0'], ['sidelight', '-0.0'], ['sideline', '-0.0'], ['siden', '0.0'], ['sident', '-0.0'], ['sider', '-0.0'], ['siderable', '-0.0'], ['siderably', '-0.0'], ['sideranko', '0.0'], ['sideration', '-0.0'], ['sidere', '-0.0'], ['sidering', '0.0'], ['siderman', '0.0'], ['sidero', '0.0'], ['siderophore', '0.0'], ['sideros', '0.0'], ['sidestep', '0.0'], ['sidewalk', '0.0'], ['sideward

In [24]:
#Test on the remaining 20% of the dataset: 
#make the model predict and compare with real

print('making predictions..')
y_pred=pipe.predict(X_test)

making predictions..


In [28]:
from openpyxl import Workbook

def evaluate(true, pred):
    # Evaluate model
    accuracy = accuracy_score(true, pred)
    precision = precision_score(true, pred, average='weighted')
    recall = recall_score(true, pred, average='weighted')
    f1 = f1_score(true, pred, average='weighted')
    cm = confusion_matrix(true, pred)
 
    # Create Excel workbook
    wb = Workbook()
    ws1 = wb.active
    ws1.title = "EvalCommit"+str(version)
    
    # Write variables to sheet 1
    ws1["A1"] = "Accuracy"
    ws1["B1"] = "{:.2%}".format(accuracy)
    ws1["A2"] = "Precision"
    ws1["B2"] = "{:.2%}".format(precision)
    ws1["A3"] = "Recall"
    ws1["B3"] = "{:.2%}".format(recall)
    ws1["A4"] = "F1 Score"
    ws1["B4"] = "{:.2%}".format(f1)
    
    # Create sheet 2 and write confusion matrix
    ws2 = wb.create_sheet("Confusion Matrix"+str(version))
    for row in cm.tolist():
        ws2.append(row)

    # Add a sheet for top negative tokens
    negative_sheet = wb.create_sheet(title="Top Negative Tokens"+str(version))
    with open('top_negative_tokens_'+str(version)+'.csv', 'r') as csvfile:
        reader = csv.reader(csvfile)
        next(reader)  # skip header row
        sorted_rows = sorted(reader, key=lambda row: float(row[1]))
        negative_sheet['A1'] = "Token"
        negative_sheet['B1'] = "Coefficient"
        for idx, row in enumerate(sorted_rows):
            negative_sheet.cell(row=idx+2, column=1, value=row[0])
            negative_sheet.cell(row=idx+2, column=2, value=row[1])

    # Add a sheet for top positive tokens
    positive_sheet = wb.create_sheet(title="Top Positive Tokens"+str(version))
    with open('top_positive_tokens_'+str(version)+'.csv', 'r') as csvfile:
        reader = csv.reader(csvfile)
        next(reader)  # skip header row
        sorted_rows = sorted(reader, key=lambda row: float(row[1]), reverse=True)
        positive_sheet['A1'] = "Token"
        positive_sheet['B1'] = "Coefficient"
        for idx, row in enumerate(sorted_rows):
            positive_sheet.cell(row=idx+2, column=1, value=row[0])
            positive_sheet.cell(row=idx+2, column=2, value=row[1])
    
    # Save workbook
    wb.save("EvalCommit"+str(version)+".xlsx")
    
    # Print evaluation metrics
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"Confusion Matrix:\n{cm}")


evaluate(y_test, y_pred)


Accuracy: 0.6159
Precision: 0.6179
Recall: 0.6159
F1 Score: 0.6064
Confusion Matrix:
[[ 5  1  0  0  0  0  0  0]
 [ 0  2  1  0  0  0  0  0]
 [ 0  0  8  1  0  1  2  1]
 [ 0  1  0  8  9  2  1  1]
 [ 0  0  0  3 16  5  4  2]
 [ 0  1  0  1  9 31 19 15]
 [ 3  0  0  0  0 12 61 14]
 [ 0  0  0  0  2  3  7 63]]


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=961bc5f7-68db-4917-95e0-61c59b88476b' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>